In [1]:
import netCDF4
import matplotlib.pyplot as plt
import numpy as np 
import datetime as dt  # Python standard library datetime  module
import math as math
from mpl_toolkits.basemap import Basemap
%matplotlib inline

In [2]:
url='http://tds.hycom.org/thredds/dodsC/GLBa0.08/expt_91.1/2015/uvel'
nc=netCDF4.Dataset(url)
print nc.variables.keys()

[u'Depth', u'Y', u'X', u'MT', u'Date', u'Latitude', u'Longitude', u'u']
